<a href="https://colab.research.google.com/github/Resul-Caliskan/CV-Extraction-Ai/blob/main/CV_Extraction_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =======================
# Install required libraries
# =======================
!pip install pymupdf nltk spacy torch transformers==4.39.3 accelerate==0.27.2 --quiet





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
exit()

In [1]:
# =======================
# Imports
# =======================
import fitz  # PyMuPDF
import string
import nltk
from nltk.corpus import stopwords
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from google.colab import files

In [2]:
# =======================
# Download necessary resources
# =======================
nltk.download('stopwords')

# =======================
# Functions
# =======================
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ''
    for page in doc:
        text += page.get_text()
    return text

def preprocess_text(text):
    """Preprocess text: lowercase, remove punctuation and stopwords."""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def load_model():
    """Load the Phi-3-mini model for text generation."""
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3-mini-4k-instruct",
        device_map=device,
        torch_dtype="auto",
        trust_remote_code=True,
    )
    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
    return model, tokenizer

def extract_cv_information(cv_text):
    """Extract structured information from CV text using AI model."""
    model, tokenizer = load_model()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    prompt = f"""
    Extract the following information from this CV and fill the structure:
    {{"name":"","email":"","phone":"","skills":"","languages":"","experience":""}}

    CV Content:
    {cv_text}

    Extracted Information:
    """

    messages = [{"role": "user", "content": prompt}]
    generation_args = {
        "max_new_tokens": 700,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

def process_cv(pdf_path):
    """Process a CV PDF and extract structured information."""
    raw_text = extract_text_from_pdf(pdf_path)
    processed_text = preprocess_text(raw_text)
    structured_info = extract_cv_information(processed_text)
    return structured_info

In [4]:

# =======================
# File Upload Interface (Colab)
# =======================
print("Lütfen bir PDF dosyası yükleyin...")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Yüklenen dosya: {filename}")
    result = process_cv(filename)
    print("\n🔍 Extracted CV Information:\n")
    print(result)

Lütfen bir PDF dosyası yükleyin...


Saving Cv (1).pdf to Cv (1).pdf
Yüklenen dosya: Cv (1).pdf


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



🔍 Extracted CV Information:

 {
  "name": "Resul Calışkan",
  "email": "resulcaliskan@gmail.com",
  "phone": "0539 305 75 32",
  "skills": "AI-based problem solving, app development, AI voice cloning, React Native, TypeScript, Tailwind CSS, Node.js, Express, AI module development, Resume parsing, Job matching, Vector database creation, TF-IDF vector method, Godot engine, Rapid prototyping, AI tools, Entrepreneurship, Game development",
  "languages": "English, Turkish",
  "experience": "Baykar intern (Sep 2023 - Aug 2024), Venhancer full stack developer (Feb 2024 - Aug 2024), Kabadayı games solo (Aug 2024 - present)"
}
